<a href="https://colab.research.google.com/github/SinaLewis/GenIce/blob/main/jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

On Google Colaboratory, you must install GenIce2 and extra plugins by yourself.


In [17]:
# install GenIce2 and extra plugins from PyPI via pip.
%pip install genice2 genice2-svg genice2-cage


In [161]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Extract POSCAR format from CIF format.



In [160]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule
import numpy as np
import random

iceName = "Ice Ih"
lattice = Lattice('ice1h')
formatter = Format("cif")
water = Molecule("physical_water")
seeds = [12345,52320,74820]

zero=0.0
numDec=6

for s in seeds:
  random.seed(s)
  np.random.seed(s)
  ice = GenIce(lattice, reshape=[[1,0,0],[0,1,0],[0,0,1]]).generate_ice(formatter, water=water)
  a,b,c = ice.splitlines()[2:5]
  a=float(a.split(' ')[-1])
  b=float(b.split(' ')[-1])
  c=float(c.split(' ')[-1])

  xyz=[entry[1:5] for entry in [item.split() for item in ice.splitlines()[22:] if item]]
  xyzTemp = [atom[1:] for atom in xyz if atom[0]=='O']
  numO = len(xyzTemp)
  xyz=xyzTemp + [atom[1:] for atom in xyz if atom[0]=='H']

  with open(f"/content/drive/MyDrive/ice_structures/ice-Ih-{s}-{numO}(H2O).vasp", "w") as f:
    f.write(f'# {numO}(H2O) - {iceName}, proton disorder seed {s}; original data generated by GenIce. https://github.com/vitroid/GenIce \n' + \
    f'1.00\n    {a:.{numDec}f}    {zero:.{numDec}f}    {zero:.{numDec}f}\n    {zero:.{numDec}f}    {b:.{numDec}f}    {zero:.{numDec}f}\n' + \
    f'    {zero:.{numDec}f}    {zero:.{numDec}f}    {c:.{numDec}f}\n' + \
    f'  O    H\n  {numO}    {2*numO}\nDirect\n' + \
    '\n'.join(['    '.join(atom) for atom in xyz]))

## Generate an SVG image

Here we are using the Forms syntax of Google Colab to choose the ice type.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

ice = "DDR"  # @param ["ice1h", "ice2", "ice3", "DDR"] {allow-input: true}

lattice = Lattice(ice)
formatter = Format("svg", shadow=True, rotate="y5,x5")
# formatter  = Format("twist", svg=True,rotatex=5,rotatey=5)
img = GenIce(lattice, reshape=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]).generate_ice(formatter)

from IPython.display import SVG, display

display(SVG(img))

## Generate a PNG image

PNG renderer is also included in genice_svg package.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("CS2")
formatter = Format("png", shadow="#8881", encode=False, bg="#fff", rotate="x5,y5")
img = GenIce(lattice, reshape=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]).generate_ice(formatter)

from IPython.display import display

display(img)

## Store the image in a file

Save it in a PNG file.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice3")
formatter = Format("png", shadow="#8881", encode=True, bg="#fff", rotate="x5,y5")
img = GenIce(lattice, reshape=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]).generate_ice(formatter)

with open("ice3.png", "wb") as f:
    f.write(img)

## Analysis

Some statistical analyses are also available.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule
import numpy as np
from matplotlib import pyplot as plt
import random

lattice = Lattice("ice1c")
formatter = Format("_KG")
water = Molecule("spce")
for seed in range(1000, 1010):
    random.seed(seed)
    np.random.seed(seed)
    result = GenIce(lattice, reshape=[[10, 0, 0], [0, 10, 0], [0, 0, 10]]).generate_ice(
        formatter, water=water
    )

    kg = np.fromstring(result, dtype=float, sep=" ")
    N = kg.shape[0]
    kg = kg.reshape([N // 2, 2])
    plt.plot(kg[:, 0], kg[:, 1])

plt.xlabel("r / nm")
plt.ylabel("Kirkwood G")
plt.show()

## Logging

You may want to see the process log.

In [ ]:
from logging import getLogger, INFO, DEBUG, WARNING

logger = getLogger()
logger.setLevel(INFO)

from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice1c")
formatter = Format("gromacs")
water = Molecule("spce")
ice = GenIce(
    lattice, reshape=[[16, 0, 0], [0, 16, 0], [0, 0, 16]], signature="Jupyter test"
).generate_ice(formatter, water=water)
with open("ice1c.gro", "w") as f:
    f.write(ice)

logger.setLevel(WARNING)

## Raw data

You can access the "raw" (internal) data.

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("xFAU", rep=3)
formatter = Format("raw", stage=(1,))
raw = GenIce(lattice, signature="Aeroice 3xFAU").generate_ice(formatter)

raw

### Visualization by Plotly

Use the raw data to plot by yourself.

In [ ]:
waters = raw["reppositions"]
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=waters[:, 0],
            y=waters[:, 1],
            z=waters[:, 2],
            marker=dict(size=10),
            mode="markers",
        )
    ]
)
fig.show()

## Integration with py3Dmol.

Py3Dmol works on Google Colaboratory.

In [ ]:
%pip install py3Dmol

In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice1h_unit")
formatter = Format("gromacs")
water = Molecule("spce")
ice = GenIce(lattice, reshape=[[4, 0, 0], [0, 4, 0], [0, 0, 4]]).generate_ice(
    formatter, water=water
)

In [ ]:
import py3Dmol

view = py3Dmol.view()
view.addModel(ice, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()

## Various file formats using MDAnalysis

In [ ]:
%pip install genice2-mdanalysis
exit(0)


In [ ]:
from genice2.genice import GenIce
from genice2.plugin import Lattice, Format, Molecule

lattice = Lattice("ice1h_unit")
formatter = Format("mdanalysis")
water = Molecule("spce")

# MDAnalysis universe
universe = GenIce(lattice, reshape=[[4, 0, 0], [0, 4, 0], [0, 0, 4]]).generate_ice(
    formatter, water=water
)

# Save as a .gro file.
allatoms = universe.select_atoms("all")
allatoms.write("1h_unit.pdb")